In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#os.chdir("/kaggle/input/g-research-crypto-forecasting/")
import gresearch_crypto
import time
import traceback
from datetime import datetime
import joblib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:

path = '/kaggle/input/g-research-crypto-forecasting/'
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "example_test.csv")
asset_details = pd.read_csv(path + "asset_details.csv")
supply_train = pd.read_csv(path + "supplemental_train.csv")

In [ ]:
train.head()

In [ ]:
def extract_feature(d:pd.DataFrame):
    d['date'] = pd.to_datetime(d['timestamp'],unit='s')
    d['weekday'] = d['date'].dt.weekday
    d['hour'] = d['date'].dt.hour
    d.drop(columns=["date"],inplace=True)
    #上影線
    d['Up_shadow'] = d['High'] - np.maximum(d['Close'],d['Open'])
    #下影線
    d['Down_shadow'] = np.minimum(d['Close'],d['Open']) - d['Low']
    return d

def model_input_asset(train:pd.DataFrame,asset:int):
    #split per asset
    d = train[train['Asset_ID'] == asset_id]
    
    #sampling
    d = d.sample(frac=1)
    print(d.head())
    d = extract_feature(d)
    # Target: Residual log-returns for the asset over a 15 minute horizon.
    d.replace([np.inf, -np.inf], np.nan, inplace=True)
#     d.dropna(how='any',inplace=True)
    d = d.fillna(method="ffill")

    Y = d['Target']
    X = d.drop('Target',axis=1)
    
#     parameter_test = {
# #         "max_depth":range(9,13,1),
# #         "max_depth":range(1,8,1),
#         "max_depth":range(1,4,1),
#         "n_estimators" : range(10000,15000,1000),
#         "num_leaves":range(2,16,2)
#     }
    
    parameter = {
        "objective": "regression",  
        "n_estimators" :11000,
        "num_leaves" : 300,
        "learning_rate" : 0.09,   
        "random_seed" : 202101,
        "verbose" : 0,
        "device" : "gpu"
    }

    model = LGBMRegressor(**parameter)
#     gsearch = GridSearchCV(model,param_grid = parameter_test,scoring='r2', cv=3)
#     gsearch.fit(X,Y)
    model.fit(X,Y)
#     print(gsearch.best_score_)
#     print(gsearch.best_params_)
    return model

In [ ]:
models = {}
asset_details = asset_details.sort_values(by='Asset_ID')

In [ ]:
asset_details

In [ ]:
for index,row in asset_details.iterrows():
    asset_id = row['Asset_ID']
    asset_name = row['Asset_Name']
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    model = model_input_asset(train, asset_id)  
    models[asset_id] = model

In [ ]:
def extract_feature_test(train):
    d = train
    d['date'] = pd.to_datetime(d['timestamp'],unit='s')
    d['weekday'] = d['date'].dayofweek
    d['hour'] = d['date'].hour
    d.drop(labels=['date'],inplace=True)
    d.replace([np.inf, -np.inf], np.nan, inplace=True)
    #上影線
    d['Up_shadow'] = d['High'] - np.maximum(d['Close'],d['Open'])
    #下影線
    d['Down_shadow'] = np.minimum(d['Close'],d['Open']) - d['Low']
    return d

In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

In [ ]:
for i, (df_test, df_pred) in enumerate(iter_test):
    #print("abc")
    for j , row in df_test.iterrows():        
        if models[row['Asset_ID']] is not None:
            try:
                model = models[row['Asset_ID']]
                rowid = row['row_id']
                row.drop(labels='row_id',inplace=True)
                #print(row)
                x_test = extract_feature_test(row)
                #print(x_test)
                y_pred = model.predict(pd.DataFrame([x_test]))[0]
                row['row_id'] = rowid
                df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
            except:
                df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
                traceback.print_exc()
        else: 
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0        
    env.predict(df_pred)

In [ ]:
sub = pd.read_csv('submission.csv')

In [ ]:
sub.head()